<a href="https://colab.research.google.com/github/NQT2906/CS114.K21/blob/Case_Study/Sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Clone data from git**

In [38]:
!git clone -b Case_Study 'https://github.com/NQT2906/CS114.K21.git' Dataset

Cloning into 'Dataset'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 809 (delta 23), reused 16 (delta 1), pack-reused 753
Receiving objects: 100% (809/809), 940.04 MiB | 34.61 MiB/s, done.
Resolving deltas: 100% (78/78), done.


# **Import library**

In [39]:
import pandas as pd
import numpy as np
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Read and combine data**

In [40]:
dataset_1 = pd.read_json('https://raw.githubusercontent.com/NQT2906/CS114.K21/Sarcasm/Sarcasm/Sarcasm_Headlines_Dataset.json', lines = True)
dataset_2 = pd.read_json('https://raw.githubusercontent.com/NQT2906/CS114.K21/Sarcasm/Sarcasm/Sarcasm_Headlines_Dataset_v2.json', lines = True)

In [41]:
dataset_1

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0
...,...,...,...
26704,https://www.huffingtonpost.com/entry/american-...,american politics in moral free-fall,0
26705,https://www.huffingtonpost.com/entry/americas-...,america's best 20 hikes,0
26706,https://www.huffingtonpost.com/entry/reparatio...,reparations and obama,0
26707,https://www.huffingtonpost.com/entry/israeli-b...,israeli ban targeting boycott supporters raise...,0


In [42]:
is_sar = dataset_2.iloc[:, 0]
art = dataset_2.iloc[:, 2]

dataset_2.iloc[:, 0] = art
dataset_2.iloc[:, 2] = is_sar

dataset_2.columns = ['article_link', 'headline', 'is_sarcastic']

In [43]:
dataset_2

,article_link,headline,is_sarcastic
0,https://www.theonion.com/thirtysomething-scien...,thirtysomething scientists unveil doomsday clo...,1
1,https://www.huffingtonpost.com/entry/donna-edw...,dem rep. totally nails why congress is falling...,0
2,https://www.huffingtonpost.com/entry/eat-your-...,eat your veggies: 9 deliciously different recipes,0
3,https://local.theonion.com/inclement-weather-p...,inclement weather prevents liar from getting t...,1
4,https://www.theonion.com/mother-comes-pretty-c...,mother comes pretty close to using word 'strea...,1
...,...,...,...
28614,https://www.theonion.com/jews-to-celebrate-ros...,jews to celebrate rosh hashasha or something,1
28615,https://local.theonion.com/internal-affairs-in...,internal affairs investigator disappointed con...,1
28616,https://www.huffingtonpost.com/entry/andrew-ah...,the most beautiful acceptance speech this week...,0
28617,https://www.theonion.com/mars-probe-destroyed-...,mars probe destroyed by orbiting spielberg-gat...,1


In [44]:
dataset = pd.concat([dataset_1, dataset_2])

In [45]:
dataset

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0
...,...,...,...
28614,https://www.theonion.com/jews-to-celebrate-ros...,jews to celebrate rosh hashasha or something,1
28615,https://local.theonion.com/internal-affairs-in...,internal affairs investigator disappointed con...,1
28616,https://www.huffingtonpost.com/entry/andrew-ah...,the most beautiful acceptance speech this week...,0
28617,https://www.theonion.com/mars-probe-destroyed-...,mars probe destroyed by orbiting spielberg-gat...,1


In [46]:
dataset = dataset.drop_duplicates()

In [47]:
dataset

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0
...,...,...,...
28497,https://www.theonion.com/tyson-holds-contest-t...,tyson holds contest to let fans submit new ide...,1
28509,https://politics.theonion.com/increasingly-coc...,increasingly cocky bernie sanders announces he...,1
28520,https://www.theonion.com/cash-strapped-zuckerb...,cash-strapped zuckerberg forced to sell 11 mil...,1
28544,https://local.theonion.com/grocery-store-bar-a...,grocery store bar actually has great little ha...,1


# **Split data to train and test (0.8 train and 0.2 test)**

In [48]:
X = dataset.iloc[:, 1].values
Y = dataset.iloc[:, 2].values

In [49]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.8, random_state = 0)

# **Preprocessing**

In [50]:
corpus_train = []
for review in X_train: # Duyệt qua từng phần tử trong X_train.values
  review = re.sub('[^a-zA-Z]', ' ', review) #remove word not alphabet
  review = review.split() # split string to word
  ps = PorterStemmer()
  review = [ps.stem(word) for word in review if word not in stopwords.words('english')] # Loại bỏ các từ không có ý nghĩa
                                                                                        # Chuyển các từ về dạng nguyên mẫu
  review = ' '.join(review) # Ghép các từ sau khi xử lý lại thành 1 câu
  corpus_train.append(review)

In [51]:
corpus_test = []
for review in X_test:
  review = re.sub('[^a-zA-Z]', ' ', review) #remove word not alphabet
  review = review.split() # split string to word
  ps = PorterStemmer()
  review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
  review = ' '.join(review)
  corpus_test.append(review)

In [52]:
print(np.shape(corpus_train))
print(np.shape(corpus_test))

(22893,)
(5724,)


# **Create dictionary for word in train and test**

In [53]:
cv = CountVectorizer()
X_train = cv.fit_transform(X_train).toarray()
trainVocab = cv.vocabulary_ #export tập từ vựng

In [54]:
cv = CountVectorizer(vocabulary=trainVocab)
X_test = cv.fit_transform(X_test).toarray()

In [55]:
print(X_test.shape)
print(Y_test.shape)

(5724, 23862)
(5724,)


# **Choose model**

In [56]:
model = MultinomialNB()
model.fit(X_train, Y_train)

Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

In [57]:
print("Score of model (MultinomialNB) in train set:", model.score(X_train, Y_train))
print("Score of model (MultinomialNB) in test set:", model.score(X_test, Y_test))

print("Accuracy score of model (MultinomialNB) in train set:",accuracy_score(Y_train, Y_train_pred))
print("Accuracy score of model (MultinomialNB) in test set:", accuracy_score(Y_test, Y_test_pred))

Score of model (MultinomialNB) in train set: 0.9319879439129864
Score of model (MultinomialNB) in test set: 0.8509783368273934
Accuracy score of model (MultinomialNB) in train set: 0.9319879439129864
Accuracy score of model (MultinomialNB) in test set: 0.8509783368273934


In [58]:
from sklearn.linear_model import LogisticRegression
model_log = LogisticRegression(random_state = 0)
model_log.fit(X_train, Y_train)

Y_train_pred = model_log.predict(X_train)
Y_test_pred = model_log.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [59]:
print("Score of model (LogisticRegression) in train set:", model_log.score(X_train, Y_train))
print("Score of model (LogisticRegression) in test set:", model_log.score(X_test, Y_test))


print("Accuracy score of model (LogisticRegression) in train set:", accuracy_score(Y_train, Y_train_pred))
print("Accuracy score of model (LogisticRegression) in test set:", accuracy_score(Y_test, Y_test_pred))

Score of model (LogisticRegression) in train set: 0.9596819988642816
Score of model (LogisticRegression) in test set: 0.8422431865828093
Accuracy score of model (LogisticRegression) in train set: 0.9596819988642816
Accuracy score of model (LogisticRegression) in test set: 0.8422431865828093


# **Save model**

In [47]:
import pickle

In [48]:
filename = 'MultinomialNB_Sarcasm.sav'
pickle.dump(model, open(filename, 'wb'))

In [49]:
filename = 'Logistic_Regression_Sarcasm.sav'
pickle.dump(model_log, open(filename, 'wb'))

# **Load model and test new data**

**Import library**

In [60]:
import pandas as pd
import numpy as np
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Load model**

In [61]:
import pickle

model_log = pickle.load(open('/content/Dataset/Logistic_Regression_Sarcasm.sav', 'rb'))
model_NB = pickle.load(open('/content/Dataset/MultinomialNB_Sarcasm.sav', 'rb'))

**Load data in crawl file**

In [66]:
datareal = pd.read_csv('/content/Dataset/Headline_Case_study.csv')

X_real = datareal.iloc[:, 1].values
Y_real = datareal.iloc[:, 2].values

**Load dictionary in training set and convert it to dict**

In [67]:
dictionary = pd.read_csv('/content/Dataset/Dictionary.csv')

dictionary = dict(dictionary.values.tolist())

**Use CountVectorizer to fit_transform X_real**

In [68]:
cv = CountVectorizer(vocabulary=dictionary)
X_real = cv.fit_transform(X_real).toarray()

**Predict and Evaluate**

In [73]:
Y_real_pred_NB = model_NB.predict(X_real)

In [74]:
print("Score of model (MultinomialNB) in real set:", model_NB.score(X_real, Y_real))

Score of model (MultinomialNB) in real set: 0.6998246053620647


In [75]:
Y_real_pred_Log = model_log.predict(X_real)

In [76]:
print("Score of model (Logistic) in real set:", model_log.score(X_real, Y_real))

Score of model (Logistic) in real set: 0.7496867952894012
